<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/PyCaret_%EC%9C%A0%EC%A0%84%ED%98%95%EC%B5%9C%EB%B9%88_MIR_GWAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving GWAS_SNP(20).csv to GWAS_SNP(20).csv
Saving GWAS_SNP(50).csv to GWAS_SNP(50).csv
Saving GWAS_SNP(100).csv to GWAS_SNP(100).csv
Saving GWAS_SNP(500).csv to GWAS_SNP(500).csv
Saving GWAS_SNP(1000).csv to GWAS_SNP(1000).csv
Saving GWAS_SNP(2000).csv to GWAS_SNP(2000).csv


In [8]:
!pip install catboost --quiet

# **MIR**

In [3]:

# 📊 PyCaret 성능 비교 (표현형별 + SNP 수별 + 모델별 정리 + 조건 출력)

!pip install pycaret[full] --quiet

from pycaret.regression import setup, compare_models, pull
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 1. 데이터 로드
geno_df = pd.read_csv("유전형인코딩_최빈값.csv", index_col=0)
pheno_df = pd.read_csv("토마토_표현형.CSV", index_col=0, encoding="cp949")

# 2. SNP 파일 목록
snp_files = {
    "Top 20": "MutualInfo_SNP(20).csv",
    "Top 50": "MutualInfo_SNP(50).csv",
    "Top 100": "MutualInfo_SNP(100).csv",
    "Top 500": "MutualInfo_SNP(500).csv",
    "Top 1000": "MutualInfo_SNP(1000).csv",
    "Top 2000": "MutualInfo_SNP(2000).csv"
}

# 3. 결과 저장용 리스트
all_results = []

# 4. 표현형별 분석
for label, file_path in snp_files.items():
    snp_df = pd.read_csv(file_path)

    for trait in snp_df["표현형"].unique():
        try:
            top_snps = snp_df[snp_df["표현형"] == trait]["SNP"].tolist()
            geno_subset = geno_df.loc[top_snps].T
            geno_subset["target"] = pheno_df[trait]

            # PyCaret 설정
            setup(data=geno_subset, target="target", session_id=42, verbose=False)

            # 모델 비교
            models_result = compare_models(include=["rf", "xgboost", "lightgbm", "catboost"], n_select=4)
            result_df = pull()
            result_df["SNP 개수"] = label
            result_df["표현형"] = trait

            print(f"📌 표현형: {trait} | SNP 수: {label}")
            display(result_df)

            all_results.append(result_df)

        except Exception as e:
            print(f"⚠️ 오류 발생 - 표현형: {trait}, SNP: {label}, 원인: {e}")
            continue

# 5. 결과 통합 및 저장
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df.to_csv("모델성능비교_전체정리.csv", index=False)
    print("✅ 결과 저장 완료 → 모델성능비교_전체정리.csv")
    display(final_df.head(10))
else:
    print("❌ 저장할 결과가 없습니다.")


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,32.2376,1996.5824,43.5662,0.2830,0.5949,0.6249,0.1470
rf,Random Forest Regressor,31.5247,2016.6453,43.1651,0.2642,0.5450,0.5996,1.2060
catboost,CatBoost Regressor,32.1777,2175.1766,44.4923,0.2353,0.5584,0.6158,0.9980
xgboost,Extreme Gradient Boosting,32.9891,2451.1569,47.4869,0.0875,0.6193,0.5861,0.1480


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

📌 표현형: 과중 (g) | SNP 수: Top 20


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
lightgbm,Light Gradient Boosting Machine,32.2376,1996.5824,43.5662,0.2830,0.5949,0.6249,0.147,Top 20,과중 (g)
rf,Random Forest Regressor,31.5247,2016.6453,43.1651,0.2642,0.5450,0.5996,1.206,Top 20,과중 (g)
catboost,CatBoost Regressor,32.1777,2175.1766,44.4923,0.2353,0.5584,0.6158,0.998,Top 20,과중 (g)
xgboost,Extreme Gradient Boosting,32.9891,2451.1569,47.4869,0.0875,0.6193,0.5861,0.148,Top 20,과중 (g)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,6.8101,97.7857,9.2357,0.1722,0.1943,0.1561,0.0810
catboost,CatBoost Regressor,6.9580,106.5563,9.7393,0.0068,0.2087,0.1650,0.4990
rf,Random Forest Regressor,6.8153,108.2396,9.7268,-0.0212,0.2035,0.1589,0.2790
xgboost,Extreme Gradient Boosting,7.7560,139.2455,11.1272,-0.3593,0.2352,0.1814,0.0740


...

Processing:   0%|          | 0/24 [00:00<?, ?it/s]

📌 표현형: 과실경도 (kg) | SNP 수: Top 2000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
catboost,CatBoost Regressor,0.0758,0.0097,0.0961,0.3435,0.0600,0.1319,39.849,Top 2000,과실경도 (kg)
rf,Random Forest Regressor,0.0774,0.0101,0.0975,0.3211,0.0606,0.1342,3.025,Top 2000,과실경도 (kg)
lightgbm,Light Gradient Boosting Machine,0.0812,0.0106,0.1004,0.2667,0.0623,0.1402,1.298,Top 2000,과실경도 (kg)
xgboost,Extreme Gradient Boosting,0.0892,0.0133,0.1125,0.0772,0.0701,0.1541,1.943,Top 2000,과실경도 (kg)


✅ 결과 저장 완료 → 모델성능비교_전체정리.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,Light Gradient Boosting Machine,32.2376,1996.5824,43.5662,0.2830,0.5949,0.6249,0.147,Top 20,과중 (g)
1,Random Forest Regressor,31.5247,2016.6453,43.1651,0.2642,0.5450,0.5996,1.206,Top 20,과중 (g)
2,CatBoost Regressor,32.1777,2175.1766,44.4923,0.2353,0.5584,0.6158,0.998,Top 20,과중 (g)
3,Extreme Gradient Boosting,32.9891,2451.1569,47.4869,0.0875,0.6193,0.5861,0.148,Top 20,과중 (g)
4,Light Gradient Boosting Machine,6.8101,97.7857,9.2357,0.1722,0.1943,0.1561,0.081,Top 20,과장 (mm)
5,CatBoost Regressor,6.9580,106.5563,9.7393,0.0068,0.2087,0.1650,0.499,Top 20,과장 (mm)
6,Random Forest Regressor,6.8153,108.2396,9.7268,-0.0212,0.2035,0.1589,0.279,Top 20,과장 (mm)
7,Extreme Gradient Boosting,7.7560,139.2455,11.1272,-0.3593,0.2352,0.1814,0.074,Top 20,과장 (mm)
8,Extreme Gradient Boosting,8.5119,135.8516,11.5584,0.3327,0.2230,0.1813,0.063,Top 20,과폭 (mm)
9,CatBoost Regressor,8.6574,137.9697,11.6340,0.3264,0.2251,0.1856,0.676,Top 20,과폭 (mm)


In [4]:
import pandas as pd

# 1. 기존 결과 불러오기
df = pd.read_csv("모델성능비교_전체정리.csv")

# 2. 표현형 + SNP 수별 R² 최고 모델만 추출
ranking_df = df.loc[df.groupby(["표현형", "SNP 개수"])["R2"].idxmax()]

# 3. 결과 저장
ranking_df.to_csv("모델성능_랭킹요약.csv", index=False)

# 4. 미리보기
ranking_df.head()


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
68,Random Forest Regressor,0.0804,0.0107,0.1003,0.2806,0.0624,0.1389,0.318,Top 100,과실경도 (kg)
116,CatBoost Regressor,0.0748,0.0097,0.0962,0.3360,0.0599,0.1293,19.746,Top 1000,과실경도 (kg)
20,Random Forest Regressor,0.0872,0.0117,0.1055,0.1992,0.0658,0.1505,0.181,Top 20,과실경도 (kg)
140,CatBoost Regressor,0.0758,0.0097,0.0961,0.3435,0.0600,0.1319,39.849,Top 2000,과실경도 (kg)
44,CatBoost Regressor,0.0793,0.0104,0.0997,0.2802,0.0623,0.1382,1.232,Top 50,과실경도 (kg)


In [6]:
# 📌 Colab 전용 성능 요약 코드 (R² 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_전체정리.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["R2"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 표현형별 최고 R² 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 R²")
display(mean_r2_by_snp)

print("📌 모델별 평균 R²")
display(mean_r2_by_model)


📌 표현형별 최고 R² 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,CatBoost Regressor,0.0758,0.0097,0.0961,0.3435,0.0600,0.1319,39.849,Top 2000,과실경도 (kg)
1,Light Gradient Boosting Machine,5.7569,70.3503,7.8234,0.4187,0.1653,0.1317,0.175,Top 100,과장 (mm)
2,CatBoost Regressor,25.6246,1311.7937,35.3588,0.5245,0.5089,0.5454,9.438,Top 500,과중 (g)
3,CatBoost Regressor,7.2578,97.2336,9.8149,0.5169,0.1947,0.1582,37.421,Top 2000,과폭 (mm)
4,Random Forest Regressor,0.5406,0.5345,0.7205,0.1547,0.1281,0.1207,0.326,Top 100,과피두께 (mm)
5,CatBoost Regressor,0.6257,0.6770,0.8103,0.3680,0.1587,0.1585,42.721,Top 2000,당도 (%)


📌 SNP 개수별 평균 R²


,SNP 개수,평균 R²
0,Top 100,0.258529
1,Top 1000,0.245846
2,Top 20,0.114496
3,Top 2000,0.281112
4,Top 50,0.159371
5,Top 500,0.263721


📌 모델별 평균 R²


,Model,평균 R²
0,CatBoost Regressor,0.277503
1,Extreme Gradient Boosting,0.098653
2,Light Gradient Boosting Machine,0.249336
3,Random Forest Regressor,0.256558


# **GWAS**

In [10]:
# 📊 PyCaret 성능 비교 (GWAS 기반 SNP + 표현형 이름 매핑)

!pip install pycaret[full] --quiet

from pycaret.regression import setup, compare_models, pull
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 1. 데이터 로드
geno_df = pd.read_csv("유전형인코딩_최빈값.csv", index_col=0)
pheno_df = pd.read_csv("토마토_표현형.CSV", index_col=0, encoding="cp949")

# 2. 표현형 이름 매핑
trait_map = {
    "과중": "과중 (g)",
    "과장": "과장 (mm)",
    "과폭": "과폭 (mm)",
    "과피두께": "과피두께 (mm)",
    "당도": "당도 (%)",
    "과실경도": "과실경도 (kg)"
}

# 3. GWAS SNP 파일 목록
snp_files = {
    "Top 20": "GWAS_SNP(20).csv",
    "Top 50": "GWAS_SNP(50).csv",
    "Top 100": "GWAS_SNP(100).csv",
    "Top 500": "GWAS_SNP(500).csv",
    "Top 1000": "GWAS_SNP(1000).csv",
    "Top 2000": "GWAS_SNP(2000).csv"
}

# 4. 결과 저장용 리스트
all_results = []

# 5. 표현형별 분석
for label, file_path in snp_files.items():
    snp_df = pd.read_csv(file_path)

    for trait in snp_df["Trait"].unique():
        try:
            mapped_trait = trait_map[trait]  # 이름 매핑

            top_snps = snp_df[snp_df["Trait"] == trait]["SNP"].tolist()
            geno_subset = geno_df.loc[top_snps].T
            geno_subset["target"] = pheno_df[mapped_trait]

            # PyCaret 설정
            setup(data=geno_subset, target="target", session_id=42, verbose=False)

            # 모델 비교
            models_result = compare_models(include=["rf", "xgboost", "lightgbm", "catboost"], n_select=4)
            result_df = pull()
            result_df["SNP 개수"] = label
            result_df["표현형"] = mapped_trait

            print(f"📌 표현형: {mapped_trait} | SNP 수: {label} (GWAS 기반)")
            display(result_df)

            all_results.append(result_df)

        except Exception as e:
            print(f"⚠️ 오류 발생 - 표현형: {trait}, SNP: {label}, 원인: {e}")
            continue

# 6. 결과 통합 및 저장
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df.to_csv("모델성능비교_전체정리_GWAS.csv", index=False)
    print("✅ 결과 저장 완료 → 모델성능비교_전체정리_GWAS.csv")
    display(final_df.head(10))
else:
    print("❌ 저장할 결과가 없습니다.")


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,27.0842,1568.6315,38.9633,0.4323,0.5249,0.5430,0.2050
rf,Random Forest Regressor,26.8466,1561.9644,38.9080,0.3977,0.4901,0.4801,1.0210
catboost,CatBoost Regressor,27.8847,1666.9463,40.0957,0.3543,0.5104,0.5024,0.6990
xgboost,Extreme Gradient Boosting,28.4679,1721.6323,40.5120,0.3387,0.5091,0.4598,0.0770


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

📌 표현형: 과중 (g) | SNP 수: Top 20 (GWAS 기반)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
lightgbm,Light Gradient Boosting Machine,27.0842,1568.6315,38.9633,0.4323,0.5249,0.5430,0.205,Top 20,과중 (g)
rf,Random Forest Regressor,26.8466,1561.9644,38.9080,0.3977,0.4901,0.4801,1.021,Top 20,과중 (g)
catboost,CatBoost Regressor,27.8847,1666.9463,40.0957,0.3543,0.5104,0.5024,0.699,Top 20,과중 (g)
xgboost,Extreme Gradient Boosting,28.4679,1721.6323,40.5120,0.3387,0.5091,0.4598,0.077,Top 20,과중 (g)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,6.4295,77.3827,8.5191,0.2941,0.1765,0.1451,0.1180
rf,Random Forest Regressor,6.5732,82.8071,8.8039,0.2293,0.1837,0.1497,0.1990
catboost,CatBoost Regressor,6.5689,80.9726,8.7367,0.2209,0.1805,0.1494,0.4790
xgboost,Extreme Gradient Boosting,7.2230,97.3635,9.5325,0.0421,0.1937,0.1615,0.1190


...

Processing:   0%|          | 0/24 [00:00<?, ?it/s]

📌 표현형: 당도 (%) | SNP 수: Top 2000 (GWAS 기반)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
rf,Random Forest Regressor,0.6321,0.6710,0.8050,0.3563,0.1549,0.1549,2.405,Top 2000,당도 (%)
catboost,CatBoost Regressor,0.6911,0.7573,0.8573,0.2880,0.1640,0.1709,44.406,Top 2000,당도 (%)
lightgbm,Light Gradient Boosting Machine,0.6851,0.7819,0.8743,0.2804,0.1661,0.1694,0.962,Top 2000,당도 (%)
xgboost,Extreme Gradient Boosting,0.7161,0.7962,0.8828,0.2591,0.1690,0.1762,2.378,Top 2000,당도 (%)


✅ 결과 저장 완료 → 모델성능비교_전체정리_GWAS.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,Light Gradient Boosting Machine,27.0842,1568.6315,38.9633,0.4323,0.5249,0.5430,0.205,Top 20,과중 (g)
1,Random Forest Regressor,26.8466,1561.9644,38.9080,0.3977,0.4901,0.4801,1.021,Top 20,과중 (g)
2,CatBoost Regressor,27.8847,1666.9463,40.0957,0.3543,0.5104,0.5024,0.699,Top 20,과중 (g)
3,Extreme Gradient Boosting,28.4679,1721.6323,40.5120,0.3387,0.5091,0.4598,0.077,Top 20,과중 (g)
4,Light Gradient Boosting Machine,6.4295,77.3827,8.5191,0.2941,0.1765,0.1451,0.118,Top 20,과장 (mm)
5,Random Forest Regressor,6.5732,82.8071,8.8039,0.2293,0.1837,0.1497,0.199,Top 20,과장 (mm)
6,CatBoost Regressor,6.5689,80.9726,8.7367,0.2209,0.1805,0.1494,0.479,Top 20,과장 (mm)
7,Extreme Gradient Boosting,7.2230,97.3635,9.5325,0.0421,0.1937,0.1615,0.119,Top 20,과장 (mm)
8,Random Forest Regressor,7.7440,110.1698,10.2339,0.4792,0.1943,0.1601,0.168,Top 20,과폭 (mm)
9,Light Gradient Boosting Machine,7.9875,110.0598,10.2892,0.4780,0.1991,0.1694,0.060,Top 20,과폭 (mm)


In [13]:
import pandas as pd

# 1. 기존 결과 불러오기
df = pd.read_csv("모델성능비교_전체정리_GWAS.csv")

# 2. 표현형 + SNP 수별 R² 최고 모델만 추출
ranking_df = df.loc[df.groupby(["표현형", "SNP 개수"])["R2"].idxmax()]

# 3. 결과 저장
ranking_df.to_csv("모델성능_gwas랭킹요약.csv", index=False)

# 4. 미리보기
ranking_df.head()


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
64,Light Gradient Boosting Machine,0.0845,0.0111,0.1045,0.2082,0.0650,0.1454,0.296,Top 100,과실경도 (kg)
112,Random Forest Regressor,0.0792,0.0111,0.1018,0.2516,0.0635,0.1361,1.436,Top 1000,과실경도 (kg)
16,Light Gradient Boosting Machine,0.0838,0.0109,0.1034,0.2341,0.0643,0.1442,0.199,Top 20,과실경도 (kg)
136,Random Forest Regressor,0.0787,0.0109,0.1011,0.2530,0.0632,0.1367,2.763,Top 2000,과실경도 (kg)
40,Light Gradient Boosting Machine,0.0852,0.0112,0.1052,0.2073,0.0654,0.1471,0.072,Top 50,과실경도 (kg)


In [12]:
# 📌 Colab 전용 성능 요약 코드 (R² 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_전체정리_GWAS.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["R2"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 표현형별 최고 R² 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 R²")
display(mean_r2_by_snp)

print("📌 모델별 평균 R²")
display(mean_r2_by_model)


📌 표현형별 최고 R² 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,Random Forest Regressor,0.0787,0.0109,0.1011,0.2530,0.0632,0.1367,2.763,Top 2000,과실경도 (kg)
1,Light Gradient Boosting Machine,5.8006,69.1458,7.9440,0.3915,0.1645,0.1305,0.089,Top 50,과장 (mm)
2,Random Forest Regressor,24.6334,1189.8322,34.2476,0.5067,0.4867,0.5392,2.962,Top 2000,과중 (g)
3,Random Forest Regressor,7.3700,100.6667,9.8816,0.5024,0.1951,0.1612,2.751,Top 2000,과폭 (mm)
4,Random Forest Regressor,0.5456,0.5661,0.7336,0.1858,0.1299,0.1219,0.708,Top 500,과피두께 (mm)
5,Random Forest Regressor,0.6321,0.6710,0.8050,0.3563,0.1549,0.1549,2.405,Top 2000,당도 (%)


📌 SNP 개수별 평균 R²


,SNP 개수,평균 R²
0,Top 100,0.157721
1,Top 1000,0.253517
2,Top 20,0.267392
3,Top 2000,0.274271
4,Top 50,0.163171
5,Top 500,0.268142


📌 모델별 평균 R²


,Model,평균 R²
0,CatBoost Regressor,0.250925
1,Extreme Gradient Boosting,0.118386
2,Light Gradient Boosting Machine,0.278617
3,Random Forest Regressor,0.274881
